# Assignment 1: Dungeons & Dragons & Distances & NaNs

**Weight**: This assignment is worth 20% of the module. 

**Grading**: Submit only a `.ipynb` file. Each part is worth 5%. Partial credit is possible for incomplete solutions. Marks may be subtracted for over-complex, unreadable, or inconsistently-formatted code. Comments should be used where needed. Your job is to write code to pass the doctests and in a few places to write a comment in a markdown cell. The doctests must not be removed or changed (it's ok to add doctests, if you wish). I have indicated how long my code is in some places. You don't have to write code the same length. It's just intended as a clue: if my code is 3 lines and yours is 30 lines, you might be doing it in a difficult way.

**Due date**: as announced on Canvas.

**Groups**: you may work in a group of 1 (ie solo) or a group of 2 of your choice. If in a group of 2, you must inform the lecturer of the group by email, cc-ing both members and including both members' names and ID numbers. You must send that email 2 weeks before the due date. And you must work together on all parts: you cannot divide the parts up between the group members. If in a group of 2, both students should make a submission and both should be identical.

**Academic integrity**: you must submit only your own work. You may discuss the assignment with other students/groups, but may not show your work to others or allow others to see yours. You may use snippets of code sourced from the internet to solve specific sub-parts, but not entire solutions. You must include a citation and URL in such cases. You may not use code generators.

**Interviews**: some students/groups will be interviewed to check on their understanding of their submission. An inability to explain your work may result in a grade penalty, a zero grade, or an academic integrity report.

**Student name(s)**: Vikrant Singh Jamwal

**Student ID(s)**: 23104534

### Part 1: Party of Adventurers

You are in a tavern in Port Blacksand. There are lots of customers. You want to choose a party of 4 (not including yourself) to go on an adventure. Each possible adventurer has various properties, eg:

```python
customers = {
    'Errik':  {'gender': 'male',   'stamina': 11, 'weapon': 'club',       'skill': 'fighting'},
    'Hider':  {'gender': 'male',   'stamina': 7,  'weapon': 'rope',       'skill': 'sneaking'},
    # etc
}   
```

Your goal is to choose a party of adventurers with the following properties:

* Exactly 4 members not including yourself 
* At least one male and at least one female
* Mean stamina at least 10
* At least two swords
* At least one person with the 'healing' skill
* Because you enjoy making up alliterative nicknames like "Torvald the Truthful", there must not be two adventurers whose names start with the same letter.

Write each of these properties as a **predicate**, ie a function that returns True or False given a `dict` of adventurers. Collect them into a list, `predicates`.

In [3]:
import doctest, math, itertools, random
import numpy as np

In [5]:
test_party = {
    'Errik':  {'gender': 'male',   'stamina': 11, 'weapon': 'sword',      'skill': 'fighting'},
    'Hider':  {'gender': 'male',   'stamina': 7,  'weapon': 'rope',       'skill': 'sneaking'},
    'Kro':    {'gender': 'male',   'stamina': 14, 'weapon': 'fists',      'skill': 'martial arts'},
    'Cleo':   {'gender': 'female', 'stamina': 4,  'weapon': 'staff',      'skill': 'healing'},
}

In [239]:
def four_members(d): 
    return len(d) == 4

# DEFINE THE OTHER PREDICATES HERE AND CREATE THE COMPLETE LIST BELOW

def gender_balance(d):
    male=sum(1 for name in d if d[name]['gender']=='male')    
    return male>0 and male<4
    # TRUE if 0<male<4, so all 4 cant be male or female.

def mean_stamina(d):
    total_stamina = sum(d[name]['stamina'] for name in d)
    return total_stamina/len(d)>=10
    #TRUE if average stamina of 4 is >=10 (float value, hence not be equivalent to >9)

def two_swords(d):
    swords=sum(1 for name in d if d[name]['weapon']=='sword')
    return swords>1
    #True if swords in the team > 1 (As it will be an integer, hence equivalent to >=2)
    
def healing(d):
    healings = sum(1 for name in d if d[name]['skill']=='healing')
    return healings>0
    #True if healings in the team > 0 (As it will be an integer, hence equivalent to >=1)

def alliteration(d):
    initials= [name[:1] for name in d]
    return len(initials)==len(set(initials))
    #TRUE if all the initials are unique, as set length = list length

predicates = [
    four_members, gender_balance, mean_stamina, two_swords, healing, alliteration
]

In [169]:
def test_preds(party, preds):
    """
    Prove that the proposed party passes all predicates.
    
    This function is complete: you don't have to write anything.
    
>>> test_preds(test_party, predicates)
four_members True
gender_balance True
mean_stamina False
two_swords False
healing True
alliteration True
    """
    for pred in preds:
        print(pred.__name__, pred(party))

In [171]:
doctest.run_docstring_examples(test_preds, globals(), verbose=True)

Finding tests in NoName
Trying:
    test_preds(test_party, predicates)
Expecting:
    four_members True
    gender_balance True
    mean_stamina False
    two_swords False
    healing True
    alliteration True
ok


Below is the complete data on the customers. Write a function using your predicates to see if there are any possible subsets of customers who could form your party.

In [222]:
customers = {
    'Errik':  {'gender': 'male',   'stamina': 11, 'weapon': 'club',       'skill': 'fighting'},
    'Hider':  {'gender': 'male',   'stamina': 7,  'weapon': 'rope',       'skill': 'sneaking'},
    'Kro':    {'gender': 'male',   'stamina': 14, 'weapon': 'fists',      'skill': 'martial arts'},
    'Cleo':   {'gender': 'female', 'stamina': 4,  'weapon': 'staff',      'skill': 'healing'},
    'Hirso':  {'gender': 'male',   'stamina': 9,  'weapon': 'keys',       'skill': 'lockpicking'},
    'Marsha': {'gender': 'female', 'stamina': 8,  'weapon': 'bow',        'skill': 'healing'},
    'Muriel': {'gender': 'female', 'stamina': 5,  'weapon': 'sword',      'skill': 'illusions'},
    'Nina':   {'gender': 'female', 'stamina': 9,  'weapon': 'crossbow',   'skill': 'sniping'},    
    'Hector': {'gender': 'male',   'stamina': 12, 'weapon': 'shield',     'skill': 'defense'},
    'Tasha':  {'gender': 'female', 'stamina': 6,  'weapon': 'flute',      'skill': 'healing'},
    'Vance':  {'gender': 'male',   'stamina': 15, 'weapon': 'hook',       'skill': 'swashbuckling'},
    'Cassie': {'gender': 'female', 'stamina': 11, 'weapon': 'whip',       'skill': 'beast taming'},
    'Loki':   {'gender': 'male',   'stamina': 4,  'weapon': 'dagger',     'skill': 'illusion'},
    'Mulan':  {'gender': 'female', 'stamina': 12, 'weapon': 'javelin',    'skill': 'strategy'},
    'Arya':   {'gender': 'female', 'stamina': 7,  'weapon': 'sword',      'skill': 'assassination'},
    'Raj':    {'gender': 'male',   'stamina': 6,  'weapon': 'sling',      'skill': 'healing'}, 
    'Duncan': {'gender': 'male',   'stamina': 7,  'weapon': 'axe',        'skill': 'woodcutting'},
    'Eve':    {'gender': 'female', 'stamina': 11, 'weapon': 'broomstick', 'skill': 'witchery'},
    'Evern':  {'gender': 'female', 'stamina': 9,  'weapon': 'sword',      'skill': 'throwing'},
    'Omar':   {'gender': 'male',   'stamina': 9,  'weapon': 'mace',       'skill': 'healing'},
    'Hamish': {'gender': 'male',   'stamina': 5,  'weapon': 'boomerang',  'skill': 'retrieval'},
    'Helena': {'gender': 'female', 'stamina': 6,  'weapon': 'sword',      'skill': 'climbing'},
    'Tyrion': {'gender': 'male',   'stamina': 5,  'weapon': 'crossbow',   'skill': 'diplomacy'},
}

In [305]:
def choose_party(customers, size, predicates):
    # Just print out each party as a tuple of names
    ## YOUR CODE HERE - mine is 5 lines
     
    
    # In one line
    
    '''---ROUGH WORK---
    
    print([party.keys() for party in map(dict, itertools.combinations(customers.items(),size)) 
           if all([predicate(party) for predicate in predicates])])
    
    '''
    
    # IN 4 LINES (clear code)
    
    parties= list(map(dict, itertools.combinations(customers.items(),size)))
    
    #parties will be a list of dictionaries of customers combinations.
    #print(parties[:10])
    
    for party in parties:
        if all([predicate(party) for predicate in predicates]):
            print(tuple(party.keys()))
            
    # As predicates argument is dict type, so parties every party will be a dict as
    # combinations of customer.items are mapped to dict type.

Now we can call `choose_party`:

In [308]:
choose_party(customers, 4, predicates)

('Vance', 'Arya', 'Evern', 'Omar')


How many different possible parties did you find? Answer by writing the number below.

### YOUR ANSWER HERE: 1 - ('Vance', 'Arya', 'Evern', 'Omar')


### Part 2: Treasure Chests

While wandering in the fortress of the Goblin King, you've discovered a box of treasure!

Your job is to tell the other adventurers **whether it contains** a `golden key`. 

The problem is - the box contains some items, like keys and coins and bracelets, but it also contains some other containers, like jewel-encrusted cases, leather pouches, or even other boxes. For each container you might have to search within it - recursively. An item is represented by a `str`, but a container is represented by a `list`.
 
Write a recursive function to return True or False.

In [313]:
def search_for(container, item):
    """
    >>> search_for(['coin', 'crown', 'key'], 'key')
    True
    >>> search_for(['coin', 'crown', 'key'], 'dagger')
    False
    >>> box = ['coin', 'coin', 'silver coin', 'silver key', 'needle', 'golden crown', 
    ...        ['coin', 'coin', 'coin'], 'diamond', 'small diamond',
    ...        ['coin'], 
    ...        ['potion of healing', 'scroll'],
    ...        ['golden key']]
    >>> search_for(box, 'coin')    
    True
    >>> search_for(box, 'silver crown')    
    False
    >>> search_for(box, 'small diamond')    
    True
    >>> search_for(box, 'potion of healing')
    True
    >>> search_for(box, 'golden key')
    True
    """
    ## YOUR CODE HERE - mine is 8 lines
    
    x= item in container
    y= False
    
    for objects in container:
        if type(objects)==list:
            y= search_for(objects,item)
            if y==True:
                break
                
    return x or y

In [315]:
doctest.run_docstring_examples(search_for, globals(), verbose=True)

Finding tests in NoName
Trying:
    search_for(['coin', 'crown', 'key'], 'key')
Expecting:
    True
ok
Trying:
    search_for(['coin', 'crown', 'key'], 'dagger')
Expecting:
    False
ok
Trying:
    box = ['coin', 'coin', 'silver coin', 'silver key', 'needle', 'golden crown', 
           ['coin', 'coin', 'coin'], 'diamond', 'small diamond',
           ['coin'], 
           ['potion of healing', 'scroll'],
           ['golden key']]
Expecting nothing
ok
Trying:
    search_for(box, 'coin')    
Expecting:
    True
ok
Trying:
    search_for(box, 'silver crown')    
Expecting:
    False
ok
Trying:
    search_for(box, 'small diamond')    
Expecting:
    True
ok
Trying:
    search_for(box, 'potion of healing')
Expecting:
    True
ok
Trying:
    search_for(box, 'golden key')
Expecting:
    True
ok


### Part 3: The NaNs of Truth

One of the items you discovered in the box of treasure was a scroll. When you read it, you see that written on it are 4 strange symbols:

In [318]:
scroll = [-math.inf, math.pi, math.inf, math.nan]

As you watch, these symbols almost seem to `float` into the air:

In [321]:
for x in scroll:
    print(type(x))

<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>


These symbols have unexpected properties which we will investigate by making **two** tables. In each table, every symbol has to be compared to every other:

* A **less than** table
* An **equal to** table. 

In a table like this, each element is either `True` or `False`, represented as `1` or `0`.

For example, a **less than** table for the `int`s `[1, 2, 3]` would be like this:

```python
0 1 1
0 0 1
0 0 0
```

Your job is to make a **less than** table and an **equal to** table for the symbols of the `scroll`.

According to the famous wizards Hunt and Thomas' **rule of DRY**, we should avoid repeating code. So, our two tables should each be created by calling two common underlying functions, one to make a table and one to print a table.

In [324]:
def print_table(table):
    """
    The input table should be a list of list of bool.
    
    This function should print the table in a nice square
    binary format.

>>> print_table([[True, True], [False, False]])
1 1
0 0
>>> print_table([[True, True, True], [False, False, False]])
1 1 1
0 0 0
    """
    ## YOUR CODE HERE - mine is 2 lines
    
    for bool_list in table:
        print(" ".join(map(str,bool_list)).replace("True","1").replace("False","0") )

In [326]:
doctest.run_docstring_examples(print_table, globals(), verbose=True)

Finding tests in NoName
Trying:
    print_table([[True, True], [False, False]])
Expecting:
    1 1
    0 0
ok
Trying:
    print_table([[True, True, True], [False, False, False]])
Expecting:
    1 1 1
    0 0 0
ok


In [327]:
def make_table(L, fn):
    """
    This function should return a list of list of bool.
>>> make_table([1, 2, 3], (lambda x, y: x < y))
[[False, True, True], [False, False, True], [False, False, False]]
    """
    ## YOUR CODE HERE - mine is 1 line
    
    ''' -----Rough Space-----
    outer_lst=[]
    for i1 in L:
        inner_lst=[]
        for i2 in L:
            if fn(i1,i2):
                inner_lst.append(True)
            else:
                inner_lst.append(False)
        outer_lst.append(inner_lst)      
    return outer_lst
    '''
   
    return [[fn(i1,i2) for i2 in L] for i1 in L ]

In [330]:
doctest.run_docstring_examples(make_table, globals(), verbose=True)

Finding tests in NoName
Trying:
    make_table([1, 2, 3], (lambda x, y: x < y))
Expecting:
    [[False, True, True], [False, False, True], [False, False, False]]
ok


In [332]:
def make_lt_table(L):
    """
>>> make_lt_table([1, 2, 3])
0 1 1
0 0 1
0 0 0
>>> scroll = [-math.inf, math.pi, math.inf, math.nan]
>>> make_lt_table(scroll)
0 1 1 0
0 0 1 0
0 0 0 0
0 0 0 0
    """
    ## YOUR CODE HERE - MUST CALL print_table and make_table
    ## mine is 1 line
    
    return print_table(make_table(L, (lambda x,y: x<y)))

In [334]:
doctest.run_docstring_examples(make_lt_table, globals(), verbose=True)

Finding tests in NoName
Trying:
    make_lt_table([1, 2, 3])
Expecting:
    0 1 1
    0 0 1
    0 0 0
ok
Trying:
    scroll = [-math.inf, math.pi, math.inf, math.nan]
Expecting nothing
ok
Trying:
    make_lt_table(scroll)
Expecting:
    0 1 1 0
    0 0 1 0
    0 0 0 0
    0 0 0 0
ok


In [336]:
def make_eq_table(L):
    """
>>> make_eq_table([1, 2, 3])
1 0 0
0 1 0
0 0 1
>>> scroll = [-math.inf, math.pi, math.inf, math.nan]
>>> make_eq_table(scroll)
1 0 0 0
0 1 0 0
0 0 1 0
0 0 0 0
    """
    ## YOUR CODE HERE - must call print_table and make_table
    ## mine is 1 line
    
    return print_table(make_table(L, (lambda x,y: x==y)))

In [338]:
doctest.run_docstring_examples(make_eq_table, globals(), verbose=True)

Finding tests in NoName
Trying:
    make_eq_table([1, 2, 3])
Expecting:
    1 0 0
    0 1 0
    0 0 1
ok
Trying:
    scroll = [-math.inf, math.pi, math.inf, math.nan]
Expecting nothing
ok
Trying:
    make_eq_table(scroll)
Expecting:
    1 0 0 0
    0 1 0 0
    0 0 1 0
    0 0 0 0
ok


The oldest member of your adventuring party then tells a long, rambling story while everyone yawns.  "In a city I don't want to mention the name of, on a street called Diagon Alley, there is a certain interesting property. Usually, in a square table like this, we see there is a special value on the diagonal. For example in an **equal to** table, usually every element is equal to itself, so we usually have `1` on the diagonal." 

But is this true for the strange symbols of the scroll? Answer by writing below the name of the strange symbol which is not equal to itself.

### YOUR ANSWER HERE: NAN symbol

### Part 4: the Nearest Defender

There is one final challenge. The evil warlock has lifted your party into a higher dimension and is going to attack. 

When he attacks, you need to find out which of you is closest to him, in order to defend. But how can you calculate that in a higher dimension (specifically, in $n$ dimensions)? Use the **Euclidean distance** spell:

$$\mathrm{distance}(x, y) = \sqrt{\sum_{i=1}^n (x_i - y_i)^2}$$

But make sure you use **vectorisation** in your spell, because otherwise your spell will be too slow when $n$ is large!

In [343]:
def distance(x, y):
    ## YOUR CODE HERE - must use vectorisation
    ## mine is 1 line
    
    return math.sqrt(sum(np.square(np.array(x)-np.array(y))))

Then, write a spell to calculate which member of your party is the closest to the attacker's location.

In [370]:
def choose_defender(defender_names, defender_locs, attacker_loc):
    """
    >>> names = ['a', 'b', 'c', 'd']
    >>> locs = [[-10, -10], [-5, 0], [5, 5], [2, 5]]
    >>> choose_defender(names, locs, [1, 4])
    'd'
    >>> choose_defender(names, locs, [-1, -14])
    'a'
    >>> locs = [[1, 1, 1, 1], [3, 1, 1, 1], [0, 0, 0, 0], [5, 5, 5, 5]]
    >>> choose_defender(names, locs, [4, 0, 0, 0])
    'b'
    """
    ## YOUR CODE HERE - mine is 5 lines
    
    distance_dict = {distance(defender_loc, attacker_loc):idx 
                     for idx, defender_loc in enumerate(defender_locs)}
    return defender_names[distance_dict[min(distance_dict)]]

    # distance_dict is the dict of all the distances as the key and the index of location as values.
    # returns the name from the index of the location.


In [372]:
doctest.run_docstring_examples(choose_defender, globals(), verbose=True)

Finding tests in NoName
Trying:
    names = ['a', 'b', 'c', 'd']
Expecting nothing
ok
Trying:
    locs = [[-10, -10], [-5, 0], [5, 5], [2, 5]]
Expecting nothing
ok
Trying:
    choose_defender(names, locs, [1, 4])
Expecting:
    'd'
ok
Trying:
    choose_defender(names, locs, [-1, -14])
Expecting:
    'a'
ok
Trying:
    locs = [[1, 1, 1, 1], [3, 1, 1, 1], [0, 0, 0, 0], [5, 5, 5, 5]]
Expecting nothing
ok
Trying:
    choose_defender(names, locs, [4, 0, 0, 0])
Expecting:
    'b'
ok
